In [1]:
#Separate processing script for DDT 17. 

#search for import above the current directory
import sys
import os
from pathlib import Path
cwd = Path().resolve().parent

sys.path.insert(1, os.path.join(cwd, 'Read Process FT Stat'))

import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt

import dataAnalyzer_FTStat
import dataScreen_FTStat

#Folder with FT Statistic-ified files. All the files need to be processed using the same metrics.
fragKey = '235'
cullTimes = {
             #'20241027_235_40':(19, 40) run once with this to determine bounds, then fill in and repeat
             #'20241027_235_17':(19.23, 24.62)}
             '20241027_235_17':(19., 30)}

allFragmentDict = {'20241027_235_17':{'235':['Unsub','13C','37Cl','37Cl-13C','37Cl-37Cl','37Cl-37Cl-13C']}}

outDict = {}
allMergedDict = {}
for folderPath, thisCullTimes in cullTimes.items():
    fragmentDict = allFragmentDict[folderPath]

    fragmentMostAbundant = ['Unsub']

    massStr = []
    fragmentIsotopeList = []
    for i, v in fragmentDict.items():
        massStr.append(i)
        fragmentIsotopeList.append(v)
        
    cullByTime = True
    cullTimes = [thisCullTimes]

    rtnAllFilesDF, mergedDict, allOutputDict = dataAnalyzer_FTStat.calc_Folder_Output(folderPath, cullOn="TIC*IT",cullAmount = 3,
                                                cullByTime=True, cullTimes = cullTimes, 
                                                fragmentIsotopeList = fragmentIsotopeList, 
                                                fragmentMostAbundant = fragmentMostAbundant, debug = True, fileExt = '.txt', 
                                                massStrList = list(fragmentDict.keys()),
                                                Microscans = 1,
                                                minNL_over_maxNL = 0.2,
                                                cull_on_IT = 3000)
    
    dataScreen_FTStat.zeroCountsScreen(mergedDict, fragmentDict[fragKey])
    dataScreen_FTStat.RSESNScreen(allOutputDict)
    
    outDict[folderPath] = rtnAllFilesDF
    allMergedDict[folderPath] = copy.deepcopy(mergedDict)

20241014_45_DDT_25uM_RES_235_1.txt
20241014_46_DDT_25uM_RES_235_17_1.txt
File 1 235 37Cl-37Cl-13C/Unsub fails RSE/SN Test with value of 11.074218681518625


In [2]:
sampleOutputDict = dataAnalyzer_FTStat.folderOutputToDict(rtnAllFilesDF)

In [3]:
# Extract subKeys for plotting
file_keys = list(sampleOutputDict.keys())
sub_keys = list(sampleOutputDict[file_keys[0]][fragKey].keys())

storedValues = {}

for i, sub_key in enumerate(sub_keys):
    # Select the data for one subKey ('13C/Unsub') across all files
    averages = [sampleOutputDict[file]['235'][sub_key]['Average'] for file in sampleOutputDict]
    rel_std_errors = [sampleOutputDict[file]['235'][sub_key]['RelStdError'] for file in sampleOutputDict]

    # Indices for samples
    sample_indices = [1]

    # Initialize lists for standardized values and propagated errors
    standardized_deltas = []
    propagated_errors = []

    # Function to calculate the propagated error
    def calculate_propagated_error(sample_error, std1_error, std2_error):
        return np.sqrt(sample_error**2 + 1/ np.sqrt(2) * (std1_error/2 + std2_error/2)**2)

    # Iterate through sample indices to calculate standardized values and errors
    for idx in sample_indices:
        sample_value = averages[idx]
        std1_value = averages[idx - 1]
        
        standardized_value = sample_value / std1_value
        standardized_delta = 1000 * (standardized_value-1)
        
        sample_error = rel_std_errors[idx]
        std1_error = rel_std_errors[idx - 1]
        
        propagated_error = np.sqrt(sample_error**2 + std1_error**2)
        propagated_error*= 1000
        
        standardized_deltas.append(standardized_delta)
        propagated_errors.append(propagated_error)
        print(propagated_errors)

    storedValues[sub_key] = {'Deltas': copy.deepcopy(standardized_deltas), 'Errors':copy.deepcopy(propagated_errors)}

[1.1268786339840913]
[1.345400522292812]
[15.638362062758732]
[1.2952192551624773]
[0.6776216510052516]


In [4]:
import json 

#Get json output

folder_key = '20241027_235_17'
output_dict = {folder_key: {}}

for sub_key, values in storedValues.items():
    deltas = values['Deltas']
    errors = values['Errors']
    
    mean_delta = float(np.mean(deltas))
    reproducibility = float(np.std(deltas, ddof=0))  # Will be 0.0 for 1 value

    output_dict[folder_key][sub_key] = {
        'Deltas': deltas,
        'Propagated Acquisition Errors': errors,
        'Mean Delta': mean_delta,
        'Experimental Reproducibility': reproducibility,
        'Mean Experimental Reproducibility': reproducibility
    }

# Save to JSON
with open('20241027_235_17_results.json', 'w') as f:
    json.dump(output_dict, f, indent=4)